# Подготовка

Установка необходимых библиотек.

In [ ]:
!pip3 install opencv-python

Импорт необходимых библиотек.

In [ ]:
import os
import random

import numpy as np
from matplotlib import pyplot as plt
import cv2

import torch
from torch import nn

import hashlib

Определяем, есть ли у нас доступ к вычислениям на GPU.

In [ ]:
use_cuda = torch.cuda.is_available()

Задаём функцию для того, чтобы зафиксировать случайность в рамках блокнота. Это необходимо для того, чтобы можно было проверить полученные решения через платформу.

In [ ]:
def set_seed(seed: int):
    """
    Фиксирует случайность в процессе обучения и применения нейронных сетей.
    
    Функция нужна для того, чтобы вне зависимости от того,
    на каком устройстве и в какой момент времени
    запускается текущий код, он всегда возвращал
    одни и те же значения.
    
    В первую очередь это нужно для того, чтобы можно было
    проверить корректность решения заданий из блокнота.
    """
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Пролог

В рамках последующих заданий мы будем работать с генератором и дискриминатором генеративно-состязательной сети, обученной на данных датасета MNIST.

Датасет содержит изображения рукописных цифр от $0$ до $9$ и уже встречался нам раньше. Например, в практической лекции в этом модуле.

Важно, что модели уже предобучены, то есть дополнительно обучать мы их не будем. Будем только применять.

Также важно, что значения пикселей в изображениях из датасета в рамах заданий мы будем рассматривать как числа в диапазоне от ${-1}$ до $1$.

# Задание 1. Смешивание шумов

Зададим функцию, с помощью которой будем генерировать входные изображения-шумы для генератора.

Для простоты будем рассматривать чёрно-белое изображение как вектор, полученный в результате конкатенации строчек матрицы изображения.

In [ ]:
def sample_noise_batch(batch_size, latent_dim):
    """
    Генерирует батч случайного шума заданной размерности.
    """
    noise_batch = torch.randn(batch_size, latent_dim)
    if use_cuda:
        noise_batch = noise_batch.cuda()
    return noise_batch

In [ ]:
class ConvGenerator(nn.Module):
    def __init__(
        self,
        latent_dim: int
    ):
        # Фиксируем случайность, чтобы при инициализации модель
        # всегда получалась одной и той же.
        set_seed(39)
        
        super().__init__()

        # Генератор принимает изображение-шум в формате вектора,
        # преобразует его с помощью линейного слоя,
        # после этого полученный вектор приводится к формату изображения HxWxC,
        # где H — высота (равна 4), W — ширина (равна 4) и C — число каналов (равно 512).
        # Далее к полученному тензору применяется ряд свёрточных слоёв,
        # чередующихся с операцией upsampling.
        # В результате получается изображение размера 28x28, в котором каждый пиксель
        # представлен числом в диапазоне от -1 до 1: за это отвечает функция активации tanh.
        self.layers = nn.Sequential(
            nn.Linear(latent_dim, 4 * 4 * 512, bias=False),
            nn.BatchNorm1d(4 * 4 * 512),
            nn.LeakyReLU(0.2),
            nn.Unflatten(1, (512, 4, 4)),

            nn.Conv2d(512, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Upsample(scale_factor=1.75, mode='nearest'),

            nn.Conv2d(256, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Upsample(scale_factor=2, mode='nearest'),

            nn.Conv2d(128, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Upsample(scale_factor=2, mode='nearest'),

            nn.Conv2d(64, 1, kernel_size=3, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.layers(x)

Задаём размер вектора шума.

In [ ]:
latent_dim = 100

Создаём генератор и подгружаем в него предобученные веса из файла `generator.pt`.

In [ ]:
generator = ConvGenerator(latent_dim=latent_dim)
state_dict = torch.load('generator.pt', map_location='cpu')
generator.load_state_dict(state_dict)
if use_cuda:
    generator = generator.cuda()

Как вы знаете, генератор для заданного случайного шума создаст новое изображение. В нашем случае это будет какая-то цифра. Для разных случайных шумов генератор может создать разные цифры.

Интересно, что если у нас есть два шума, соответствующих разным цифрам, мы можем смешать их, чтобы получить в результате генерации нечто среднее. В рамках задания мы пронаблюдаем этот эффект.

Сначала сгенерируем два разных шума, соответствующих разным цифрам.

In [ ]:
set_seed(108)
noise_1 = sample_noise_batch(1, latent_dim)
noise_2 = sample_noise_batch(1, latent_dim)

Шумы можно смешивать в разных пропорциях. Эти пропорции будут задаваться коэффициентом $\alpha$.

Формула для смешивания шумов:
$$\text{noise_mixed} = \alpha \cdot \text{noise_1} + (1 - \alpha) \cdot \text{noise_2}.$$

Зададим несколько вариантов коэффициента $\alpha$, чтобы посмотреть как будет меняться сгенерированное изображение в зависимости от объёма примеси того или иного шума в шуме, на основе которого происходит генерация.

In [ ]:
alphas = [0.0, 0.25, 0.5, 0.75, 1.]

Сгенерируем изображения на основе шума, полученного с помощью смешения двух других шумов в разных пропорциях.

**Обратите внимание,** что в коде ниже вам необходимо самостоятельно заполнить переменную `noise_mixed` в теле цикла. Переменная заполняется согласно формуле получения смешенного шума, зафиксированной выше.

In [ ]:
gen_images = []

generator.eval()
for alpha in alphas:
    with torch.no_grad():
        noise_mixed = ... # TODO
        gen_image = generator(noise_mixed)
        gen_images.append(gen_image)


Посмотрим на сгенерированные изображения.

In [ ]:
plt.figure(figsize=(10, 4))
for i in range(len(alphas)):
    gen_image_np = gen_images[i][0].cpu().permute(1, 2, 0).numpy()
    plt.subplot(1, len(alphas), i + 1)
    plt.imshow(gen_image_np, vmin=-1, vmax=1., cmap='Greys_r')
    plt.axis('off')
    plt.title(f'alpha={alphas[i]}')

Видим, что по мере изменения соотношения шума от разных цифр в шуме, который используется для генерации, генерируемое изображение плавно переходит от цифры «4» к цифре «6».

Это ровно тот эффект, который мы и хотели пронаблюдать.

> **Ввод ответа на платформе. Вопрос 1.**
>
> Ниже приведён код, который превратит последнее сгенерированное изображение в хэш-код. В качестве ответа на задание необходимо указать тот хэш, который будет выведен на экран в результате исполнения ячейки ниже.

In [ ]:
gen_image_test = gen_images[-1][0].cpu().permute(1, 2, 0).numpy()
hash_test = hashlib.sha256(gen_image_test.data.tobytes()).hexdigest()
print(hash_test)

В следующем задании нам понадобится полученное изображение с цифрой «6», сохраним его.

In [ ]:
torch.save(gen_images[-1][0].cpu().permute(1, 2, 0), 'generated.pt')

# Задание 2. Странное поведение дискриминатора

В рамках задания у нас уже есть предобученный дискриминатор, который умеет определять правдоподобие изображений из датасета MNIST. Для того чтобы им воспользоваться, нужно воссоздать соответствующую архитектуру в коде с помощью готовых слоёв из PyTorch.

In [ ]:
class ConvDiscriminator(nn.Module):
    def __init__(self):
        # Фиксируем случайность, чтобы при инициализации модель
        # всегда получалась одной и той же.
        set_seed(42)
        
        super().__init__()

        # Дискриминатор имеет классическую свёрточную архитектуру:
        # несколько свёрточных слоёв, извлекающих признаки,
        # после которых следуют линейные слои, которые отвечают
        # непосредственно за классификацию объекта.
        self.layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1, stride=2),  # 14x14
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=2),  # 7x7
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=2),  # 4x4
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Flatten(start_dim=1, end_dim=-1),
            nn.Linear(256 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

Создаём дискриминатор с заданной архитектурой и подгружаем в него предобученные веса из файла `discriminator.pt`.

In [ ]:
discriminator = ConvDiscriminator()
state_dict = torch.load('discriminator.pt', map_location='cpu')
discriminator.load_state_dict(state_dict)
if use_cuda:
    discriminator = discriminator.cuda()

Фиксируем размер изображений, которые будем подавать на вход дискриминатору, — $28 \times 28$, что соответствует размеру изображения из датасета MNIST.

In [ ]:
image_size = 28

Создадим изображение-шум, формат которого будет совпадать с форматом изображений из MNIST.

In [ ]:
set_seed(42)
noise = torch.randn((1, 1, image_size, image_size), dtype=torch.float32)

if use_cuda:
    noise = noise.cuda()

Оценим с помощью дискриминатора вероятность того, что данный шум является правдоподобным изображением из MNIST.

In [ ]:
discriminator.eval()
with torch.no_grad():
    disc_on_noise = discriminator(noise)
    
noise_np = noise[0].cpu().permute(1, 2, 0).numpy()
disc_on_noise_item = disc_on_noise.item()

Выведем сам шум и вероятность, которую ему сопоставил дискриминатор.

In [ ]:
plt.imshow(noise_np, interpolation='none', cmap='Greys_r');
plt.title(f'{disc_on_noise_item:.4f}')
plt.xticks([])
plt.yticks([]);

> **Ввод ответа на платформе. Вопрос 1.**
>
> В качестве ответа на 1-й вопрос введите вероятность, которую дискриминатор назначил сгенерированному выше шуму.
>
> Ответ округлите до $4$ знаков после запятой.

Наблюдаем странное поведение для уже обученного дискриминатора. Обычному шуму он назначил большую вероятность существования.

Попробуем сгенерировать ещё одно изображение-шум. В этот раз будем разрешать генерацию либо строго чёрных, либо строго белых пикселей.

In [ ]:
set_seed(43)
rand_bin = torch.randint(2, size=(1, 1, image_size, image_size), dtype=torch.float32) * 2 - 1

if use_cuda:
    rand_bin = rand_bin.cuda()

Снова оцениваем вероятность правдоподобия, которую дискриминатор присвоил нашему шуму.

In [ ]:
discriminator.eval()
with torch.no_grad():
    disc_on_rand_bin = discriminator(rand_bin)
    
rand_bin_np = rand_bin[0].cpu().permute(1, 2, 0).numpy()
disc_on_rand_bin_item = disc_on_rand_bin.item()

In [ ]:
plt.imshow(rand_bin_np, interpolation='none', cmap='Greys_r', vmin=-1., vmax=1.);
plt.title(f'{disc_on_rand_bin_item:.4f}')
plt.xticks([])
plt.yticks([]);

Видим, что дискриминатор снова ошибся, посчитав шум правдоподобным изображением. Закрадываются сомнения в корректной работе этого дискриминатора на сгенерированных генератором примерах.

Давайте проверим, как он справляется с ними. Для этого загрузим уже готовый сгенерированный пример с цифрой «6» и оценим его правдоподобие с точки зрения дискриминатора.

In [ ]:
generated = torch.load('generated.pt').reshape(1, 28, 28, 1).permute(0, 3, 1, 2)

if use_cuda:
    generated = generated.cuda()
    
discriminator.eval()
with torch.no_grad():
    disc_on_generated = discriminator(generated)
    
generated_np = generated[0].cpu().permute(1, 2, 0).numpy()
disc_on_generated_item = disc_on_generated.item()

In [ ]:
plt.imshow(generated_np, interpolation='none', cmap='Greys_r', vmin=-1., vmax=1.);
plt.title(f'{disc_on_generated_item:.4f}')
plt.xticks([])
plt.yticks([]);

А для сгенерированного генератором примера дискриминатор не ошибся, выдав его вероятность правдоподобия в районе $5\%$.

> **Ввод ответа на платформе. Вопрос 2.**
>
> В качестве ответа на 2-й вопрос введите вероятность, которую дискриминатор назначил сгенерирвоанной цифре «4».
>
> Ответ округлите до $4$ знаков после запятой.

Почему получается так, что для очевидно некорректных примеров, которые являются шумом, дискриминатор выдаёт высокую вероятность правдоподобия, а похожим на правдоподобные изображениям, сгенерированным генератором, назначает низкую вероятность правдоподобия?

Дискриминатор лишь учится отличать сгенерированные примеры от настоящих (при этом на поздних шагах обучения ему надо только уметь отличать менее качественные примеры рукописных цифр от более качественных).

Таким образом, дискриминатор может не иметь ни малейшего представления о том, как устроен весь остальной мир (все остальное пространство изображений) и насколько правдоподобными являются наблюдения, которые не получены в результате генерации, а просто находятся в других частях пространства изображений.

Убедимся в этом, продемонстрировав дискриминатору изображение, аналоги которого он точно не мог видеть в процессе обучения, — изображение кота.

In [ ]:
cat = cv2.imread('cat.jpeg')
cat = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
cat = cv2.resize(cat, (28, 28))
cat = torch.tensor(cat, dtype=torch.float32) / 127.5 - 1
cat = cat.view(1, 1, 28, 28)
if use_cuda:
    cat = cat.cuda()

In [ ]:
discriminator.eval()
with torch.no_grad():
    disc_on_cat = discriminator(cat)
    
cat_np = cat[0].cpu().permute(1, 2, 0).numpy()
disc_on_cat_item = disc_on_cat.item()

In [ ]:
plt.imshow(cat_np, interpolation='none', cmap='Greys_r', vmin=-1., vmax=1.);
plt.title(f'{disc_on_cat_item:.4f}')
plt.xticks([])
plt.yticks([]);

Видим, что дискриминатор считает изображение кота очень правдоподобным. Однако это связано не с тем, что оно действительно существует, а с тем, что в процессе обучения дискриминатор никогда таких изображений не видел и, соответственно, никак не учился с ними работать. А значит, может назначить им любую произвольную вероятность.